In [16]:
import cv2
import mediapy as media
import numpy as np

def load_video(file_path, image_size=None, original_fps=30, new_fps=5, start_time=None, end_time=None, gray=False):
    """Loads a video file into a TF tensor."""
    cap = cv2.VideoCapture(file_path)
    
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    ret = True

    if image_size is None:
        image_size = (frameWidth, frameHeight)
    
    if start_time and end_time:
        start_frame = original_fps * start_time
        end_frame = original_fps * end_time
    else:
        start_frame = 0
        end_frame = frameCount
        
    fc = 0    
    
    fps_factor = original_fps / new_fps
    frame_loc = 0
    now_frame = 0

    # print(end_frame, start_frame, fps_factor)
    # print(int((end_frame - start_frame) / fps_factor))

    if gray:
        buf = np.zeros((int((end_frame - start_frame) / fps_factor), image_size[1], image_size[0]), np.dtype('uint8'))
    else:
        buf = np.zeros((int((end_frame - start_frame) / fps_factor), image_size[1], image_size[0], 3), np.dtype('uint8'))

    while (fc < end_frame - start_frame  and ret):
        ret, tmp = cap.read()
        now_frame += 1
        frame_loc += 1
        if start_frame > now_frame:
            continue
        if end_frame < now_frame:
            break
        if frame_loc > fps_factor:
            tmp = cv2.resize(tmp, dsize=image_size)
            if gray:
                buf[fc] = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
            else:
                buf[fc] = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
            fc += 1
            frame_loc -= fps_factor
    cap.release()
    
    return buf

In [89]:
media.write_video('demo/sample_result0001.mp4', video, fps=30)